In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import re

def extract_all_invoices(html_file_path):
    """
    Extrai todas as invoices do HTML, independentemente do tipo (Supplier ou não),
    e sem excluir nenhum Period Allocation Account.
    """
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    columns = [
        'Period Allocation Account',
        'Created from',
        'Voucher text',
        'Original amount',
        'Account',
        'Period allocation amount',
        'Period',
        'Sum completed',
        'Sum not completed'
    ]
    all_data = []

    data_rows = soup.find_all('tr', attrs={'valign': 'top', 'style': 'word-wrap:break-word;'})

    for row in data_rows:
        cells = row.find_all('td')
        if len(cells) < 9:
            continue

        # Pega o Period Allocation Account associado à linha
        parent = row
        account_number = None
        for _ in range(10):
            parent = parent.find_parent()
            if parent:
                b_tag = parent.find('b', string=re.compile(r'Period allocation account:\s*\d+'))
                if b_tag:
                    match = re.search(r'Period allocation account:\s*(\d+)', b_tag.get_text())
                    if match:
                        account_number = int(match.group(1))
                        break

        # Extrair os dados da linha
        created_from = extract_created_from(cells[0])

        # Ignorar invoices com número < 14728
        invoice_number_match = re.search(r'(\d+)', created_from)
        if invoice_number_match and int(invoice_number_match.group(1)) < 14728:
            continue

        voucher_text = extract_voucher_text_from_cell(cells[1])
        original_amount = clean_amount(cells[2].get_text(strip=True))
        account = clean_text(cells[3].get_text(strip=True))
        period_alloc = clean_amount(cells[4].get_text(strip=True))
        period = clean_text(cells[5].get_text(strip=True))
        sum_completed = clean_amount(cells[7].get_text(strip=True))
        sum_not_completed = clean_amount(cells[8].get_text(strip=True))

        all_data.append([
            account_number,
            created_from,
            voucher_text,
            original_amount,
            account,
            period_alloc,
            period,
            sum_completed,
            sum_not_completed
        ])

    return pd.DataFrame(all_data, columns=columns)

# Funções auxiliares
def extract_created_from(cell):
    link = cell.find('a')
    if link:
        created_from = link.get_text().strip()
    else:
        created_from = cell.get_text().strip()
    return created_from.split('\n')[0].strip()

def extract_voucher_text_from_cell(cell):
    cell_html = str(cell).replace('<br>', ' | ').replace('<br/>', ' | ')
    temp_soup = BeautifulSoup(cell_html, 'html.parser')
    return temp_soup.get_text().strip()

def clean_amount(text):
    return text.replace('\xa0', '').replace(' ', '').strip()

def clean_text(text):
    return text.replace('\xa0', '').strip()

# Execução
if __name__ == "__main__":
    html_file = '/Users/yngird.figlioli/Library/CloudStorage/OneDrive-Mews/Desktop/Atomize/report.html'
    df = extract_all_invoices(html_file)

    if not df.empty:
        output_path = 'all_invoices.csv'
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"✅ Dados salvos em {output_path}")
    else:
        print("⚠️ Nenhum dado encontrado.")


✅ Dados salvos em all_invoices.csv


In [3]:
df

,Period Allocation Account,Created from,Voucher text,Original amount,Account,Period allocation amount,Period,Sum completed,Sum not completed
0,2970,Customer invoice 122577180,Periodisering av kundfaktura 12257 | Hotel Sct...,"67305,60",3001,"5608,80",2024-06-01 - 2025-05-31,"61696,80","5608,80"
1,1790,Customer invoice 122587394,Periodisering av kundfaktura 12258 | Makarem U...,"102638,67",3002,"8553,25",2024-06-01 - 2025-05-31,"94085,42","8553,25"
2,1790,Customer invoice 122597378,Periodisering av kundfaktura 12259 | THE YAMU ...,"65662,43",3002,"5471,86",2024-06-01 - 2025-05-31,"60190,57","5471,86"
3,1790,Customer invoice 123627204,Periodisering av kundfaktura 12362 | Mollie’s ...,"36597,26",3002,"3049,79",2024-06-01 - 2025-05-31,"33547,47","3049,79"
4,1790,Customer invoice 123637205,Periodisering av kundfaktura 12363 | Mollie’s ...,"56929,07",3002,"4744,08",2024-06-01 - 2025-05-31,"52184,99","4744,08"
...,...,...,...,...,...,...,...,...,...
708,1790,Customer invoice 144321391,Period allocation of customer invoice 14432 | ...,"3251,18",3001,"3251,18",2025-06-01 - 2025-06-30,"0,00","3251,18"
709,1790,Customer invoice 144331387,Period allocation of customer invoice 14433 | ...,"2446,54",3002,"2446,54",2025-06-01 - 2025-06-30,"0,00","2446,54"
710,1790,Customer invoice 144341393,Period allocation of customer invoice 14434 | ...,"5436,75",3001,"5436,75",2025-06-01 - 2025-06-30,"0,00","5436,75"
711,1790,Customer invoice 144351379,Period allocation of customer invoice 14435 | ...,"2157,63",3002,"2157,63",2025-06-01 - 2025-06-30,"0,00","2157,63"


In [4]:
df_invoice = pd.DataFrame()

In [5]:
df_invoice['Invoice No.'] = df['Created from']
df_invoice['Parent/Customer No.'] = df['Account']
df_invoice['Description'] = df['Voucher text']
df_invoice['Unit Price Excl. VAT'] = df['Period allocation amount']
df_invoice['CUSTOMER Dimension'] = df['Account']
df_invoice['No.'] = "RMS PACKAGE"
df_invoice['Quantity'] = "1"

In [6]:
# Extraindo as datas da coluna 'period'
start_date, end_date = df['Period'][0].split(' - ')

# Adicionando as datas extraídas ao DataFrame df_invoices
df_invoice['Deferral Start Date'] = df['Period'].apply(lambda x: x.split(' - ')[0])
df_invoice['Deferral End Date'] = df['Period'].apply(lambda x: x.split(' - ')[1])

In [7]:
df_invoice

,Invoice No.,Parent/Customer No.,Description,Unit Price Excl. VAT,CUSTOMER Dimension,No.,Quantity,Deferral Start Date,Deferral End Date
0,Customer invoice 122577180,3001,Periodisering av kundfaktura 12257 | Hotel Sct...,"5608,80",3001,RMS PACKAGE,1,2024-06-01,2025-05-31
1,Customer invoice 122587394,3002,Periodisering av kundfaktura 12258 | Makarem U...,"8553,25",3002,RMS PACKAGE,1,2024-06-01,2025-05-31
2,Customer invoice 122597378,3002,Periodisering av kundfaktura 12259 | THE YAMU ...,"5471,86",3002,RMS PACKAGE,1,2024-06-01,2025-05-31
3,Customer invoice 123627204,3002,Periodisering av kundfaktura 12362 | Mollie’s ...,"3049,79",3002,RMS PACKAGE,1,2024-06-01,2025-05-31
4,Customer invoice 123637205,3002,Periodisering av kundfaktura 12363 | Mollie’s ...,"4744,08",3002,RMS PACKAGE,1,2024-06-01,2025-05-31
...,...,...,...,...,...,...,...,...,...
708,Customer invoice 144321391,3001,Period allocation of customer invoice 14432 | ...,"3251,18",3001,RMS PACKAGE,1,2025-06-01,2025-06-30
709,Customer invoice 144331387,3002,Period allocation of customer invoice 14433 | ...,"2446,54",3002,RMS PACKAGE,1,2025-06-01,2025-06-30
710,Customer invoice 144341393,3001,Period allocation of customer invoice 14434 | ...,"5436,75",3001,RMS PACKAGE,1,2025-06-01,2025-06-30
711,Customer invoice 144351379,3002,Period allocation of customer invoice 14435 | ...,"2157,63",3002,RMS PACKAGE,1,2025-06-01,2025-06-30


In [8]:
new_columns = [
                "Subaccount No.", "Document Date", "Posting Date", "Due Date",
                "VAT Date", "Currency Code", "Type",
                "VAT Prod. Posting Group", 
                "Deferral Code", "BU Dimension", "C Dimension", "ENTITY Dimension", "IC Dimension",
                "PRICE Dimension", "PRODUCT Dimension", "RECURRENCE Dimension",
                "SUBPRODUCT Dimension", "TAX DEDUCTIBILITY Dimension", "Reseller Code", "Apply Overpayments" ]

            # Adiciona cada coluna com valor vazio ""
for col in new_columns: df_invoice[col] = ""

In [9]:
ordered_columns = [
                "Invoice No.","Parent/Customer No.","Subaccount No.","Document Date","Posting Date","Due Date","VAT Date","Currency Code","Type",
                "No.","Description","Quantity","Unit Price Excl. VAT","VAT Prod. Posting Group","Deferral Code","Deferral Start Date","Deferral End Date",
                "BU Dimension","C Dimension","ENTITY Dimension","IC Dimension","PRICE Dimension","PRODUCT Dimension","RECURRENCE Dimension",
                "SUBPRODUCT Dimension","TAX DEDUCTIBILITY Dimension","CUSTOMER Dimension","Reseller Code","Apply Overpayments"]

            # Reordenar as colunas do DataFrame
df_invoice = df_invoice[ordered_columns]

In [10]:
df_invoice

,Invoice No.,Parent/Customer No.,Subaccount No.,Document Date,Posting Date,Due Date,VAT Date,Currency Code,Type,No.,...,ENTITY Dimension,IC Dimension,PRICE Dimension,PRODUCT Dimension,RECURRENCE Dimension,SUBPRODUCT Dimension,TAX DEDUCTIBILITY Dimension,CUSTOMER Dimension,Reseller Code,Apply Overpayments
0,Customer invoice 122577180,3001,,,,,,,,RMS PACKAGE,...,,,,,,,,3001,,
1,Customer invoice 122587394,3002,,,,,,,,RMS PACKAGE,...,,,,,,,,3002,,
2,Customer invoice 122597378,3002,,,,,,,,RMS PACKAGE,...,,,,,,,,3002,,
3,Customer invoice 123627204,3002,,,,,,,,RMS PACKAGE,...,,,,,,,,3002,,
4,Customer invoice 123637205,3002,,,,,,,,RMS PACKAGE,...,,,,,,,,3002,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,Customer invoice 144321391,3001,,,,,,,,RMS PACKAGE,...,,,,,,,,3001,,
709,Customer invoice 144331387,3002,,,,,,,,RMS PACKAGE,...,,,,,,,,3002,,
710,Customer invoice 144341393,3001,,,,,,,,RMS PACKAGE,...,,,,,,,,3001,,
711,Customer invoice 144351379,3002,,,,,,,,RMS PACKAGE,...,,,,,,,,3002,,


In [13]:
!pip install openpyxl


  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [14]:
if not df.empty:
    output_path = 'Atomize_invoice.xlsx'
    df.to_excel(output_path, index=False, engine='openpyxl')
    print(f"✅ Dados salvos em {output_path}")
else:
    print("⚠️ Nenhum dado encontrado.")


✅ Dados salvos em Atomize_invoice.xlsx


In [ ]:
##OLD VERSION

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re

def extract_period_allocations(html_file_path, exclude_account=2970):
    """
    Extrai dados de todas as Period allocation accounts, exceto a informada em `exclude_account`.

    A função percorre o HTML identificando blocos iniciados por "Period allocation account: XXXX" e extrai
    todas as linhas de dados associadas àquela conta até encontrar outra conta ou o final da página.

    Args:
        html_file_path (str): Caminho para o arquivo HTML.
        exclude_account (int): Número da conta que deve ser ignorada.

    Returns:
        pd.DataFrame: Tabela com os dados extraídos.
    """
    from bs4 import BeautifulSoup
    import re
    import pandas as pd

    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    columns = [
        'Period Allocation Account',
        'Created from',
        'Voucher text',
        'Original amount',
        'Account',
        'Period allocation amount',
        'Period',
        'Sum completed',
        'Sum not completed'
    ]
    all_data = []

    # Encontrar todos os blocos que iniciam com Period allocation account
    b_tags = soup.find_all('b', string=re.compile(r'Period allocation account:\s*\d+'))

    for b_tag in b_tags:
        match = re.search(r'Period allocation account:\s*(\d+)', b_tag.get_text())
        if not match:
            continue
        account_number = int(match.group(1))
        if account_number == exclude_account:
            continue

        # Início do bloco de dados
        current = b_tag.find_parent('tr')
        while current:
            next_tr = current.find_next_sibling('tr')
            if not next_tr:
                break

            # Se encontrar outra conta, parar o loop
            if next_tr.find('b', string=re.compile(r'Period allocation account:\s*\d+')):
                break

            # Procurar linhas de dados com o padrão esperado
            data_rows = next_tr.find_all('tr', attrs={'valign': 'top', 'style': 'word-wrap:break-word;'})
            for row in data_rows:
                cells = row.find_all('td')
                if len(cells) >= 9:
                    created_from = extract_created_from(cells[0])
                    voucher_text = extract_voucher_text_from_cell(cells[1])
                    original_amount = clean_amount(cells[2].get_text(strip=True))
                    account = clean_text(cells[3].get_text(strip=True))
                    period_alloc = clean_amount(cells[4].get_text(strip=True))
                    period = clean_text(cells[5].get_text(strip=True))
                    sum_completed = clean_amount(cells[7].get_text(strip=True))
                    sum_not_completed = clean_amount(cells[8].get_text(strip=True))

                    all_data.append([
                        account_number,
                        created_from,
                        voucher_text,
                        original_amount,
                        account,
                        period_alloc,
                        period,
                        sum_completed,
                        sum_not_completed
                    ])
            current = next_tr

    return pd.DataFrame(all_data, columns=columns)


def extract_created_from(cell):
    """
    Extrai o texto de 'Created from' que pode conter links
    """
    # Verificar se há um link
    link = cell.find('a')
    if link:
        created_from = link.get_text().strip()
    else:
        created_from = cell.get_text().strip()
    
    # Pegar apenas a primeira linha (antes do <br>)
    created_from = created_from.split('\n')[0].strip()
    return created_from

def extract_voucher_text_from_cell(cell):
    """
    Extrai o texto do voucher, que pode ter múltiplas linhas separadas por <br>
    """
    # Substituir <br> por espaço para juntar as linhas
    cell_html = str(cell)
    cell_html = cell_html.replace('<br>', ' | ').replace('<br/>', ' | ')
    
    # Criar nova soup para extrair apenas o texto
    temp_soup = BeautifulSoup(cell_html, 'html.parser')
    voucher_text = temp_soup.get_text().strip()
    
    return voucher_text

def clean_amount(text):
    """
    Limpa valores monetários, removendo espaços extras
    """
    return text.replace('\xa0', '').strip()

def clean_text(text):
    """
    Limpa texto geral, removendo espaços não-quebráveis e extras
    """
    return text.replace('\xa0', '').strip()

def save_results(df, output_file='period_allocations.csv'):
    """
    Salva o DataFrame em CSV e mostra um resumo
    """
    if not df.empty:
        # Salvar em CSV
        df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"\nDados salvos em: {output_file}")
        
        # Mostrar resumo
        print("\n=== RESUMO DOS DADOS ===")
        print(f"Total de registros: {len(df)}")
        print(f"Contas encontradas: {sorted(df['Period Allocation Account'].unique())}")
        print(f"Colunas: {list(df.columns)}")
        
        # Mostrar primeiras linhas
        print("\n=== PRIMEIRAS 3 LINHAS ===")
        print(df.head(3).to_string())
        
        return True
    else:
        print("Nenhum dado para salvar")
        return False

# Exemplo de uso
if __name__ == "__main__":
    # Caminho para seu arquivo HTML
    html_file = "/Users/yngird.figlioli/Library/CloudStorage/OneDrive-Mews/Desktop/Atomize/report.html"  # Substitua pelo nome do seu arquivo
    
    try:
        print("=== EXTRAÇÃO DE DADOS - PERIOD ALLOCATIONS ===")
        print(f"Arquivo: {html_file}")
        print(f"Excluindo conta: 2970")
        print("-" * 60)
        
        # Extrair dados
        df = extract_period_allocations(html_file, exclude_account=2970)
        
        # Salvar resultados
        if save_results(df):
            print("\n✅ Extração concluída com sucesso!")
        else:
            print("\n❌ Nenhum dado foi extraído")
            
    except FileNotFoundError:
        print(f"❌ Arquivo {html_file} não encontrado")
    except Exception as e:
        print(f"❌ Erro durante a extração: {e}")
        import traceback
        traceback.print_exc()

# Função adicional para análise
def analyze_accounts(html_file_path):
    """
    Função para analisar quais contas existem no arquivo (útil para debug)
    """
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Encontrar todas as contas
    period_accounts = soup.find_all('b', string=re.compile(r'Period allocation account:\s*\d+'))
    
    print("=== ANÁLISE DE CONTAS ENCONTRADAS ===")
    accounts = []
    for account_tag in period_accounts:
        account_text = account_tag.get_text()
        account_match = re.search(r'Period allocation account:\s*(\d+)', account_text)
        if account_match:
            account_number = int(account_match.group(1))
            accounts.append(account_number)
            print(f"Conta encontrada: {account_number}")
    
    print(f"\nTotal de contas: {len(accounts)}")
    print(f"Contas únicas: {sorted(set(accounts))}")
    
    return accounts

=== EXTRAÇÃO DE DADOS - PERIOD ALLOCATIONS ===
Arquivo: /Users/yngird.figlioli/Library/CloudStorage/OneDrive-Mews/Desktop/Atomize/report.html
Excluindo conta: 2970
------------------------------------------------------------

Dados salvos em: period_allocations.csv

=== RESUMO DOS DADOS ===
Total de registros: 1
Contas encontradas: [np.int64(1790)]
Colunas: ['Period Allocation Account', 'Created from', 'Voucher text', 'Original amount', 'Account', 'Period allocation amount', 'Period', 'Sum completed', 'Sum not completed']

=== PRIMEIRAS 3 LINHAS ===
   Period Allocation Account            Created from                                                        Voucher text Original amount Account Period allocation amount                   Period Sum completed Sum not completed
0                       1790  Supplier Invoices 1976  Periodisering av leverantörsfaktura 1976 | HubSpot Ireland Limited        6 419,79    5420                   267,52  2023-11-01 - 2025-10-31      4 814,82         